### Custom Shallow Network - miniVGG style
With batch normalization and (optional) random cropping of the image

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [2]:
import os
import datetime
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.layers import Input, Activation, Dropout, Flatten, Dense
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.utils  import to_categorical
import matplotlib.pyplot as plt 

%matplotlib inline

SyntaxError: trailing comma not allowed without surrounding parentheses (<ipython-input-2-cfe8f1fb12be>, line 4)

In [ ]:
# dimensions of our images.
img_width, img_height = 224, 224
train_data_dir = '/home/sanchit/Documents/Projects/datasets/fire_and_smoke_data/train/'
validation_data_dir = '/home/sanchit/Documents/Projects/datasets/fire_and_smoke_data/val/'
nb_train_samples = 2400
nb_validation_samples = 490
epochs = 50
batch_size = 32
model_path = "./models/cust_network.h5"

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
model = Sequential()
input_shape = (img_width, img_height, 3)
model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='he_normal', 
                 kernel_regularizer=l2(1e-4), input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3), padding='same', strides=2, kernel_initializer='he_normal', 
                 kernel_regularizer=l2(1e-4), input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='he_normal', 
                 kernel_regularizer=l2(1e-4), input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3), padding='same', strides=2, kernel_initializer='he_normal', 
                 kernel_regularizer=l2(1e-4), input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='he_normal', 
                 kernel_regularizer=l2(1e-4), input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3), padding='same', strides=2, kernel_initializer='he_normal', 
                 kernel_regularizer=l2(1e-4), input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal', 
                 kernel_regularizer=l2(1e-4), input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3), padding='same', strides=2, kernel_initializer='he_normal', 
                 kernel_regularizer=l2(1e-4), input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(128, (3, 3), padding='same', kernel_initializer='he_normal', 
                 kernel_regularizer=l2(1e-4), input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(128, (3, 3), padding='same', strides=2, kernel_initializer='he_normal', 
                 kernel_regularizer=l2(1e-4), input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(GlobalAveragePooling2D())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True, 
    vertical_flip=True, 
    brightness_range=[0.6, 1.3])

# this is the augmentation configuration we will use for testing: only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

In [ ]:
# define callbacks before starting the training
early_stop = EarlyStopping(monitor="val_loss", patience=10, mode="min", verbose=1)
model_checkpoint = ModelCheckpoint(model_path, monitor="val_acc", save_best_only=True, mode='max', verbose=1)
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard = TensorBoard(log_dir=logdir, histogram_freq=1)
callbacks = [early_stop, model_checkpoint, tensorboard]

H = model.fit_generator(train_generator, 
                        steps_per_epoch=nb_train_samples // batch_size, 
                        epochs=epochs,
                        validation_data=validation_generator,
                        validation_steps=nb_validation_samples // batch_size, 
                        callbacks = callbacks)

# launch TensorBoard
%tensorboard --logdir logs